# Quickstart

https://www.django-rest-framework.org/tutorial/quickstart/

## Project setup
- startproject is run with a . after the project name to put the app in the project directory, but it helps avoid clashes with external module's namespaces

## Serializers
`HyperlinkedModelSerializer` used instead of primary key and various other relationships - hyperlinking is good RESTful design

## Views
We group common behavior into classes called `ViewSets`.  These can be very concise and organized.

## URLs
When using viewsets, we can generate the URL conf for our API by registering viewsets with a router class in `urls.py`
- if we need more control over API URLs we can use class-based views and explicitely write URL conf
- default login and logout views are added for browsable API

## Pagination
Determines how many objects per page are returned.  Edit `settings.py`.

# API access
Run the server, then 
`curl -H 'Accept: application/json; indent=4' -u admin:password123 http://127.0.0.1:8000/users/`
or with HTTPie
`http -a admin:password123 http://127.0.0.1:8000/users/`
or point the browser to
http://127.0.0.1:8000/users/

# In-depth Tutorial
https://www.django-rest-framework.org/tutorial/1-serialization/

The tutorial walks through setting up the snippets app, and creating a model for it.

## Creating a Serializer class
We can serialize data into `json` by declaring serializers in a way similar to Django's forms

### snippets/serializers.py

In [ ]:
from rest_framework import serializers
from snippets.models import Snippet, LANGUAGE_CHOICES, STYLE_CHOICES


class SnippetSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(required=False, allow_blank=True, max_length=100)
    code = serializers.CharField(style={'base_template': 'textarea.html'})
    linenos = serializers.BooleanField(required=False)
    language = serializers.ChoiceField(choices=LANGUAGE_CHOICES, default='python')
    style = serializers.ChoiceField(choices=STYLE_CHOICES, default='friendly')

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return Snippet.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.title = validated_data.get('title', instance.title)
        instance.code = validated_data.get('code', instance.code)
        instance.linenos = validated_data.get('linenos', instance.linenos)
        instance.language = validated_data.get('language', instance.language)
        instance.style = validated_data.get('style', instance.style)
        instance.save()
        return instance

### Serializing

Now that we have this set up, run `python manage.py shell` from the terminal.  Once inside:

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser

# we'll add some data to play with
snippet = Snippet(code='foo = "bar"\n')
snippet.save()

snippet = Snippet(code='print("hello, world")\n')
snippet.save()

In [ ]:
# now we can see what the serialized output is
# this will be Python native datatypes
serializer = SnippetSerializer(snippet)
serializer.data

In [ ]:
# render to json
content = JSONRenderer().render(serializer.data)
content

### Deserialization

In [ ]:
import io

# parse a stream into Python native datatypes
stream = io.BytesIO(content)
data = JSONParser().parse(stream)

In [ ]:
# restore native datatypes to object instance
serializer = SnippetSerializer(data=data)
serializer.is_valid()
# True
serializer.validated_data
# OrderedDict([('title', ''), ('code', 'print("hello, world")\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')])
serializer.save()
# <Snippet: Snippet object>

### Serializing querysets
Add `many=True` to serializer arguments

In [ ]:
serializer = SnippetSerializer(Snippet.objects.all(), many=True)
serializer.data
# [OrderedDict([('id', 1), ('title', ''), ('code', 'foo = "bar"\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 2), ('title', ''), ('code', 'print("hello, world")\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 3), ('title', ''), ('code', 'print("hello, world")'), ('linenos', False), ('language', 'python'), ('style', 'friendly')])]

### ModelSerializers
`SnippetSerializer` and the `Snippet` model are a little redundant.  REST provides `Serializer` and `ModelSerializer` classes.  We can rewrite `snippets/serializers` by replacing `SnippetsSerializer`

In [ ]:
class SnippetSerializer(serializers.ModelSerializer):
    class Meta:
        model = Snippet
        fields = ['id', 'title', 'code', 'linenos', 'language', 'style']

`ModelSerializer` classes are a shortcut for creating serializer classes:
- automatically determined set of fields
- simple `create()` and `update()` methods are implemented for you

## Writing views using Serializer
First we'll write views as regular Django views in `snippets/views.py`

In [ ]:
from django.http import HttpResponse, JsonResponse
from django.views.decorators.csrf import csrf_exempt
from rest_framework.parsers import JSONParser
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer

Will have a view that supports listing all the existing snippets or creating a new snippet

In [ ]:
@csrf_exempt
def snippet_list(request):
    """
    List all code snippets, or create a new snippet.
    """
    if request.method == 'GET':
        snippets = Snippet.objects.all()
        serializer = SnippetSerializer(snippets, many=True)
        return JsonResponse(serializer.data, safe=False)

    elif request.method == 'POST':
        data = JSONParser().parse(request)
        serializer = SnippetSerializer(data=data)
        if serializer.is_valid():
            serializer.save()
            return JsonResponse(serializer.data, status=201)
        return JsonResponse(serializer.errors, status=400)

The view is marked as `csrf_exempt` to make things easier for now - we wouldn't do with REST framework views.  For viewing an individual snippet:

In [ ]:
@csrf_exempt
def snippet_detail(request, pk):
    """
    Retrieve, update or delete a code snippet.
    """
    try:
        snippet = Snippet.objects.get(pk=pk)
    except Snippet.DoesNotExist:
        return HttpResponse(status=404)

    if request.method == 'GET':
        serializer = SnippetSerializer(snippet)
        return JsonResponse(serializer.data)

    elif request.method == 'PUT':
        data = JSONParser().parse(request)
        serializer = SnippetSerializer(snippet, data=data)
        if serializer.is_valid():
            serializer.save()
            return JsonResponse(serializer.data)
        return JsonResponse(serializer.errors, status=400)

    elif request.method == 'DELETE':
        snippet.delete()
        return HttpResponse(status=204)

 Create `snippets/urls.py` and wire the views

In [ ]:
from django.urls import path
from snippets import views

urlpatterns = [
    path('snippets/', views.snippet_list),
    path('snippets/<int:pk>/', views.snippet_detail),
]

Edit `tutorial/urls.py`

In [ ]:
from django.urls import path, include

urlpatterns = [
    path('', include('snippets.urls')),
]

# Testing
Run `python manage.py runserver`.  In another terminal, install `httpie` with pip.  
- Run `http http://127.0.0.1:8000/snippets/`
- Run `http http://127.0.0.1:8000/snippets/2/` to reference by id
- `json` can be viewed by going through the browser:
    - http://127.0.0.1:8000/snippets/
    - http://127.0.0.1:8000/snippets/2/

# Requests and Responses
## Request objects
`Request` objects extend `HttpRequest` to give us more parsing options.

`request.data` can be used for `POST`, `PUT`, and `PATCH` methods
## Response objects
`Response` objects come from `TemplateResponse`, and uses content negotiation to return correct content type data.
## Status codes
REST framework gives more explicit identifiers for status codes like `HTTP_400_BAD_REQUEST` using the `status` module
## Wrapping API views
1. `@api_view` decorator for function-based views
1. `APIView` class for working with class-based views

Both:
- Make sure you receive `Request` instances in your view
- Add context to `Response` objects 
- Detailed responses
- Handle `ParseError` exceptions if `request.data` is accessed with flawed input

## Refactoring request objects
Replace django imports with rest_framework imports, make use of request objects to make code more concise in `snippets/views.py`.  It's very similar to working with normal Django views.

In [ ]:
from rest_framework import status
from rest_framework.decorators import api_view
from rest_framework.response import Response
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer


@api_view(['GET', 'POST'])
def snippet_list(request):
    """
    List all code snippets, or create a new snippet.
    """
    if request.method == 'GET':
        snippets = Snippet.objects.all()
        serializer = SnippetSerializer(snippets, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = SnippetSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

In [ ]:
@api_view(['GET', 'PUT', 'DELETE'])
def snippet_detail(request, pk):
    """
    Retrieve, update or delete a code snippet.
    """
    try:
        snippet = Snippet.objects.get(pk=pk)
    except Snippet.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = SnippetSerializer(snippet)
        return Response(serializer.data)

    elif request.method == 'PUT':
        serializer = SnippetSerializer(snippet, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        snippet.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

## Optional format suffixes
If our responses are no longer hardwired to a single content type, we can add support for format suffixes to our API endpoints.  This will make the URLs explicitly refer to a format.  It can be done by adding `format=None` as an argument in both views.

Update `snippets/urls.py` to have `format_suffix_patterns`

In [ ]:
from django.urls import path
from rest_framework.urlpatterns import format_suffix_patterns
from snippets import views

urlpatterns = [
    path('snippets/', views.snippet_list),
    path('snippets/<int:pk>', views.snippet_detail),
]

urlpatterns = format_suffix_patterns(urlpatterns)

Run the server and try:
`http http://127.0.0.1:8000/snippets/`

and

`http http://127.0.0.1:8000/snippets/ Accept:application/json` 

and 

`http http://127.0.0.1:8000/snippets/ Accept:text/html`

and

`http http://127.0.0.1:8000/snippets.json`

and

`http http://127.0.0.1:8000/snippets.api`

and

`http --form POST http://127.0.0.1:8000/snippets/ code="print(123)"`

and

`http --json POST http://127.0.0.1:8000/snippets/ code="print(456)"`

Note: adding `--debug` to these will show the request type in request headers

Browsable API can be used since HTML is returned by default when requested by a web browser.

# Class-based views
We are going to re-write the root view as a class-based view.
> - This creates a pattern that allows us to reuse common functionality
> - It keeps our code [DRY](https://en.wikipedia.org/wiki/Don't_repeat_yourself)

Let's refactor `snippets/views.py` with the below code

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from django.http import Http404
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status


class SnippetList(APIView):
    """
    List all snippets, or create a new snippet.
    """
    def get(self, request, format=None):
        snippets = Snippet.objects.all()
        serializer = SnippetSerializer(snippets, many=True)
        return Response(serializer.data)

    def post(self, request, format=None):
        serializer = SnippetSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
class SnippetDetail(APIView):
    """
    Retrieve, update or delete a snippet instance.
    """
    def get_object(self, pk):
        try:
            return Snippet.objects.get(pk=pk)
        except Snippet.DoesNotExist:
            raise Http404

    def get(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = SnippetSerializer(snippet)
        return Response(serializer.data)

    def put(self, request, pk, format=None):
        snippet = self.get_object(pk)
        serializer = SnippetSerializer(snippet, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk, format=None):
        snippet = self.get_object(pk)
        snippet.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

Then we refactor `snippets/urls.py`.  Note `as_view()`

In [ ]:
from django.urls import path
from rest_framework.urlpatterns import format_suffix_patterns
from snippets import views

urlpatterns = [
    path('snippets/', views.SnippetList.as_view()),
    path('snippets/<int:pk>/', views.SnippetDetail.as_view()),
]

urlpatterns = format_suffix_patterns(urlpatterns)

## Using mixins
Because create/retreive/update/delete operations are similar for any model-backend API views we create.  REST framework's mixin classes provide this behavior.  Let's refactor `snippets/views.py` again using these.

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import mixins
from rest_framework import generics

class SnippetList(mixins.ListModelMixin,
                  mixins.CreateModelMixin,
                  generics.GenericAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)

    def post(self, request, *args, **kwargs):
        return self.create(request, *args, **kwargs)

View is built with `GenericAPIView` (core functionality) with `ListModelMixin` and `CreateModelMixin` (give `.list()` and `.create()` actions.  Then we bind get/post to the right actions.

In [ ]:
class SnippetDetail(mixins.RetrieveModelMixin,
                    mixins.UpdateModelMixin,
                    mixins.DestroyModelMixin,
                    generics.GenericAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

    def get(self, request, *args, **kwargs):
        return self.retrieve(request, *args, **kwargs)

    def put(self, request, *args, **kwargs):
        return self.update(request, *args, **kwargs)

    def delete(self, request, *args, **kwargs):
        return self.destroy(request, *args, **kwargs)

`GenericAPIView` class for core again, with mixins added for `.retreive()`, `.update()`, and `.destroy()` actions.

## Using generic class-based views
REST framework provides a set of already mixed-in generic views that we can use to trim down `views.py` more.

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework import generics


class SnippetList(generics.ListCreateAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer


class SnippetDetail(generics.RetrieveUpdateDestroyAPIView):
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer

Let's test this again to prove it behaves the same:

Run the server and try: `http http://127.0.0.1:8000/snippets/`

## Authentication and Permissions
Right now anyone can delete or edit code snippets.  Let's make this more secure.
- We want to associate with a creator
- only authenticated users can create
- Only the creator of a snippet may update or delete
- Unauthenticated requests should have full read-only access

To make these changes, we have to start by adding fields to our model.

### Adding info to our model
Let's add these fields to our `snippets/models.py` file:

In [ ]:
owner = models.ForeignKey('auth.User', related_name='snippets', on_delete=models.CASCADE)
highlighted = models.TextField()

Since we will use the `pygments` code-highlighting library, we'll add these imports to the top of `snippets/models.py`:

In [ ]:
from pygments.lexers import get_lexer_by_name
from pygments.formatters.html import HtmlFormatter
from pygments import highlight

We then add the `.save()` method to the model class:

In [ ]:
def save(self, *args, **kwargs):
    """
    Use the `pygments` library to create a highlighted HTML
    representation of the code snippet.
    """
    lexer = get_lexer_by_name(self.language)
    linenos = 'table' if self.linenos else False
    options = {'title': self.title} if self.title else {}
    formatter = HtmlFormatter(style=self.style, linenos=linenos,
                              full=True, **options)
    self.highlighted = highlight(self.code, lexer, formatter)
    super(Snippet, self).save(*args, **kwargs)

Instead of performing a db migration, the tutorial says to delete the db and start again (maybe this is quicker with all the fiddling we did):
```
rm -f db.sqlite3
rm -r snippets/migrations
python manage.py makemigrations snippets
python manage.py migrate
```

When done, create a superuser:
`python manage.py createsuperuser`

### Adding endpoints for User models
With a user to work with, we can create better representations in the API.  We'll make a new serializer by adding this to  `snippets/serializers.py`:

In [ ]:
from django.contrib.auth.models import User

class UserSerializer(serializers.ModelSerializer):
    snippets = serializers.PrimaryKeyRelatedField(many=True, queryset=Snippet.objects.all())

    class Meta:
        model = User
        fields = ['id', 'username', 'snippets']

The `snippets` field has a reverse relationship on the `User` model - it won't be included by default when using `ModelSerializer` so we add an explicit field for it.

Let's add some views to `snippets/views.py`.  We can use `ListAPIView` and `RetreiveAPIView` for generic class-based views:

In [ ]:
from django.contrib.auth.models import User
from snippets.serializers import UserSerializer

class UserList(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer


class UserDetail(generics.RetrieveAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer

We are using the `UserSerializer` class in both views.

Let's edit `snippets/urls.py` to get the views added to the API.  Add:

In [ ]:
path('users/', views.UserList.as_view()),
path('users/<int:pk>/', views.UserDetail.as_view()),

### Associating snippets with users
Currently the user isn't sent as part of the serialized representation - they are a property of the incoming request.  We want to associate the user that created the snippet with the snippet instance.  We can override the `.perform_create()` method on our views.  This let's us modify how the instance save is managed and handle info that is implicit in the incoming request or requested URL.

Add the code below to the `SnippetList` class in `snippets/views.py`, and the `create()` method of the serializer will be passed the `owner` filed, and the validated data from the request:

In [ ]:
def perform_create(self, serializer):
    serializer.save(owner=self.request.user)

### Updating the serializer
We can add a field to the `SnippetSerializer` definition in `snippets/seralizers.py`:

In [ ]:
class SnippetSerializer(serializers.ModelSerializer):
    owner = serializers.ReadOnlyField(source='owner.username')
    class Meta:
        model = Snippet
        fields = ['id', 'title', 'code', 'linenos', 'language', 'style', 'owner',]

The `source` argument controls the attribute that is used to populate a field, and can point to any attribute on the serialized instance.  The dotted notation used in the code below allows it to traverse given attributes in a similiar way as it is used in Django's template language.

`ReadOnlyField` is untyped, always read-only, used for serialized representations, but not used for updating model instances when they are deserialized.  `CharField(read_only=True)` could have also been used here.

### Adding required permissions to views
To ensure only authenticated users can cretae, update, and delete snippets, we can use REST framework's permission classes.  `IsAuthenticatedOrReadOnly` will make sure auth reqs get read-write access, and unathenticated requests get read-only access.

In [ ]:
# add this import to snippets/views.py
from rest_framework import permissions

Add this property to both `SnippetList` and `SnippetDetail` view classes:

In [ ]:
permission_classes = [permissions.IsAuthenticatedOrReadOnly]

### Adding login to Browsable API
We can add in the login for the browsable API by eding the URLconf in our project-level `tutorial/urls.py` file:

In [ ]:
from django.urls import path, include

# add this to bottom of file
urlpatterns += [
    path('api-auth/', include('rest_framework.urls')),
]

Run the server with `python manage.py runserver`, and navigate to http://127.0.0.1:8000/snippets/ with your browser.  Example output after submitting a snippet:

```
HTTP 201 Created
Allow: GET, POST, HEAD, OPTIONS
Content-Type: application/json
Vary: Accept

{
    "id": 1,
    "title": "All Unique",
    "code": "def all_unique(lst):\r\n    return len(lst) == len(set(lst))",
    "linenos": false,
    "language": "python",
    "style": "default",
    "owner": "brandon"
}
```

After creating a few snippets, go to http://127.0.0.1:8000/users/ and note that the snippet ids are associated with each user in each user's `snippets` field.

```
HTTP 200 OK
Allow: GET, HEAD, OPTIONS
Content-Type: application/json
Vary: Accept

[
    {
        "id": 1,
        "username": "brandon",
        "snippets": [
            1,
            2,
            3
        ]
    }
```

### Object-level permissions
We'd like the code snippets to be visible to anyone, but make sure the user who created it can update or delete it.  We'll create a custom permission.  Create a new file called `permissions.py` in the `snippets` folder.  Add the code below:

In [ ]:
from rest_framework import permissions


class IsOwnerOrReadOnly(permissions.BasePermission):
    """
    Custom permission to only allow owners of an object to edit it.
    """

    def has_object_permission(self, request, view, obj):
        # Read permissions are allowed to any request,
        # so we'll always allow GET, HEAD or OPTIONS requests.
        if request.method in permissions.SAFE_METHODS:
            return True

        # Write permissions are only allowed to the owner of the snippet.
        return obj.owner == request.user

We can add the custom permission to our snippet instance endpoint by editing the `permissions_classes` property on the `SnippetDetail` view class, and add the import for `IsOwnerOrReadOnly`:

In [ ]:
from snippets.permissions import IsOwnerOrReadOnly

permission_classes = [permissions.IsAuthenticatedOrReadOnly,
                      IsOwnerOrReadOnly]

If you go back to http://127.0.0.1:8000/snippets/1/ and note that PUT and DELETE only appear if you're logged in as the user who created the snippet.

### Authenticating with the API
If we want to edit snippets, we want to authenticate our requests.  REST will apply default `authentication classes` if we don't set them up - these are `SessionAuthentication` and `BasisAuthentication`.  Before this, we were interacting with the API through the web browser.  To authenticate programatically, we'll need to provide the authentication credentials on each request.

This can be done with this:
`http -a bob:password123 POST http://127.0.0.1:8000/snippets/ code="print(789)"`

An example output you would see would be:
```
HTTP/1.1 201 Created
Allow: GET, POST, HEAD, OPTIONS
Content-Length: 108
Content-Type: application/json
Date: Fri, 02 Oct 2020 02:25:09 GMT
Referrer-Policy: same-origin
Server: WSGIServer/0.2 CPython/3.8.5
Vary: Accept, Cookie
X-Content-Type-Options: nosniff
X-Frame-Options: DENY

{
    "code": "print(789)",
    "id": 5,
    "language": "python",
    "linenos": false,
    "owner": "bob",
    "style": "friendly",
    "title": ""
}
```

## Relationships and Hyperlinked APIs
Instead of using primary keys for relationships in our API, we can use hyperlinking for relationships.  This improves cohesion and discoverability.

### Creating an endopiont for the root of the API
While we do have endpoints for `snippets` and `users`  we don't have a single entry point for our API.  Here's how a function-based view would be implemented by adding to `snippets/views.py`:

In [ ]:
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework.reverse import reverse


@api_view(['GET'])
def api_root(request, format=None):
    return Response({
        'users': reverse('user-list', request=request, format=format),
        'snippets': reverse('snippet-list', request=request, format=format)
    })

The `reverse` function is used to return fully-qualified URls, and URL patterns are identified by convenience names we will put in `snippets/urs.py`

### Creating an endpoint for highlighted snippets
We want to use an HTML representation instead of JSON for this.  REST has two we can use - one for dealing with HTML rendered in templates, and the other dealing with pre-rendered HTML.  We'll used the 2nd.

We also need to keep in mind we're not returning an object instance, but instead a property of an object instance.  We'll use the base class for representing instances and make our own `.get()` method.  Add this to `snippets/views.py`:

In [ ]:
from rest_framework import renderers
from rest_framework.response import Response

class SnippetHighlight(generics.GenericAPIView):
    queryset = Snippet.objects.all()
    renderer_classes = [renderers.StaticHTMLRenderer]

    def get(self, request, *args, **kwargs):
        snippet = self.get_object()
        return Response(snippet.highlighted)

Update `snippets/urls.py` with the URLconf updates and the url pattern for the snippet highlights:

In [ ]:
path('', views.api_root),
path('snippets/<int:pk>/highlight/', views.SnippetHighlight.as_view()),

### Hyperlinking our API
REST framework supports all of these relationship styles, and can apply them forward or reverse across custom managers:
- Using primary keys.
- Using hyperlinking between entities.
- Using a unique identifying slug field on the related entity.
- Using the default string representation of the related entity.
- Nesting the related entity inside the parent representation.
- Some other custom representation.

To use hyperlinked style between entities, we can extend `HyperLinkedModelSerializer`.  This is different than `ModelSerializer` in that:
- it does not include the `id` field by default
- it includes a `url` field using `HyperlinkedIdentityField` instead of `PrimaryKeyRelatedField`

Replace these serializers in `snippets/seralizers.py`:

In [ ]:
class SnippetSerializer(serializers.HyperlinkedModelSerializer):
    owner = serializers.ReadOnlyField(source='owner.username')
    highlight = serializers.HyperlinkedIdentityField(view_name='snippet-highlight', format='html')

    class Meta:
        model = Snippet
        fields = ['url', 'id', 'highlight', 'owner',
                  'title', 'code', 'linenos', 'language', 'style']


class UserSerializer(serializers.HyperlinkedModelSerializer):
    snippets = serializers.HyperlinkedRelatedField(many=True, view_name='snippet-detail', read_only=True)

    class Meta:
        model = User
        fields = ['url', 'id', 'username', 'snippets']

We added a new `highlight` field, and it points to the `snippet-highlight` url pattern instead of the `snippet-detail` url.  We indicate in the `highlight` field that any format suffixed hyperlinks it returns should use the `.html` suffix.

### Making sure URL patterns are named
To have a hyperlinked API, we need to make sure we nae our URL patterns.  We need to name:
- The root of our API refers to `user-list` and `snippet-list`.
- Our snippet serializer includes a field that refers to `snippet-highlight`.
- Our user serializer includes a field that refers to `snippet-detail`.
- Our snippet and user serializers include `url` fields that by default will refer to `{model_name}-detail`, which in this case will be `snippet-detail` and `user-detail`.

The `snippets/urls.py` file will look like this:

In [ ]:
from django.urls import path
from rest_framework.urlpatterns import format_suffix_patterns
from snippets import views

# API endpoints
urlpatterns = format_suffix_patterns([
    path('', views.api_root),
    path('snippets/',
        views.SnippetList.as_view(),
        name='snippet-list'),
    path('snippets/<int:pk>/',
        views.SnippetDetail.as_view(),
        name='snippet-detail'),
    path('snippets/<int:pk>/highlight/',
        views.SnippetHighlight.as_view(),
        name='snippet-highlight'),
    path('users/',
        views.UserList.as_view(),
        name='user-list'),
    path('users/<int:pk>/',
        views.UserDetail.as_view(),
        name='user-detail')
])

### Adding pagination
If we have a lot of code snippets, we'll need to paginate the results and allow the API to step through these pages.  We can modify `tutorial/settings.py`:

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10
}

The settings in REST framework are namespaced into a single dictionary setting named `REST_FRAMEWORK`

Go to http://127.0.0.1:8000/ and click on the hyperlinks to navigate around.

## ViewSets and Routers
REST includes an absraction for `ViewSets` which allows us to concentrate on modeling the state and interactions of the API and lets URL construction happen automatically based on common conventions.

`ViewSet` classes are almost the same as `View` classes, but they have operations such as `read` or `update` instead of `get` or `put`.

`ViewSet` classes are only bound to a set of method handlers at the last moment, when it's instantiated into a set of views, usually by using a `Router` class which handles the complexities of defining the URL conf for you.

### Refactoring to use ViewSets
Let's refactor `UserList` and `UserDetail` views into a single `UserViewSet`.  We'll replace them with a single class in `snippets/views.py`:

In [ ]:
from rest_framework import viewsets

class UserViewSet(viewsets.ReadOnlyModelViewSet):
    """
    This viewset automatically provides `list` and `detail` actions.
    """
    queryset = User.objects.all()
    serializer_class = UserSerializer

`ReadOnlyModelViewSet` automatically provides default 'read-only' operations.  `queryset` and `serial_class` attributes don't need to have the same information in two classes now.  We can replace `SnippetList`, `SnippetDetail`, and `SnippetHighlight` with a single class as well:

In [ ]:
from rest_framework.decorators import action
from rest_framework.response import Response

class SnippetViewSet(viewsets.ModelViewSet):
    """
    This viewset automatically provides `list`, `create`, `retrieve`,
    `update` and `destroy` actions.

    Additionally we also provide an extra `highlight` action.
    """
    queryset = Snippet.objects.all()
    serializer_class = SnippetSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly,
                          IsOwnerOrReadOnly]

    @action(detail=True, renderer_classes=[renderers.StaticHTMLRenderer])
    def highlight(self, request, *args, **kwargs):
        snippet = self.get_object()
        return Response(snippet.highlighted)

    def perform_create(self, serializer):
        serializer.save(owner=self.request.user)

We used `ModelViewSet` to get the complete set of default read and write operations.

We use the `@action` decorator to respond to `GET` requests by default.  If we wanted an action that responded to `POST` requests, we could use the `methods` argument.

URLs for custom actions depend on the method names themselves.  f you want to change the way the URL should be constructed, you can include `url_path` as a decorator keyword argument.

### Binding ViewSets to URLs explicitely
Handler methods only get bound to actions when we define the URLConf.  If we want to see what's going on under the hood, we can explictely create a set of views from our ViewSets.

Edit `snippets/urls.py` to bind our `ViewSet` classes into a set of concrete views:

In [ ]:
from snippets.views import SnippetViewSet, UserViewSet, api_root
from rest_framework import renderers

snippet_list = SnippetViewSet.as_view({
    'get': 'list',
    'post': 'create'
})
snippet_detail = SnippetViewSet.as_view({
    'get': 'retrieve',
    'put': 'update',
    'patch': 'partial_update',
    'delete': 'destroy'
})
snippet_highlight = SnippetViewSet.as_view({
    'get': 'highlight'
}, renderer_classes=[renderers.StaticHTMLRenderer])
user_list = UserViewSet.as_view({
    'get': 'list'
})
user_detail = UserViewSet.as_view({
    'get': 'retrieve'
})

We create multiple views from each `ViewSet` class by binding the http methods to the required action for each view.  Once the're bound, we can register the views with the URLConf in `snippets/urls.py`:

In [ ]:
urlpatterns = format_suffix_patterns([
    path('', api_root),
    path('snippets/', snippet_list, name='snippet-list'),
    path('snippets/<int:pk>/', snippet_detail, name='snippet-detail'),
    path('snippets/<int:pk>/highlight/', snippet_highlight, name='snippet-highlight'),
    path('users/', user_list, name='user-list'),
    path('users/<int:pk>/', user_detail, name='user-detail')
])

### Using routers
Since we're using `ViewSet` classes rather than `View` classes, we don't need to change the URLConf ourselves.  These can be handled automatically using a `Router` class.  Once we register the appropriate view sets with a router, it will do the rest.  We can modify the `snippets/urls.py` file like so:

In [ ]:
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from snippets import views

# Create a router and register our viewsets with it.
router = DefaultRouter()
router.register(r'snippets', views.SnippetViewSet)
router.register(r'users', views.UserViewSet)

# The API URLs are now determined automatically by the router.
urlpatterns = [
    path('', include(router.urls)),
]

Registering viewsets with a router is similar to providing a urlpattern.  We used the URL preix for the views, and the viewset itself as arguments.

`DefaultRouter` automatically creates the API root view for us, so we can delete the api_root method from `snippets/views.py`

Check it out! http://127.0.0.1:8000/

### Tradoffs between views & viewsets
Viewsets are a really useful abstraction, and ensure that the URL conventions are consistent across your API, cuts down on the amount of code you need to write, and allows you to concentrate on the interactions and representations the API provides rather then the specifics in the URLConf.

It's not always the right approach to take - there are tradeoffs to consider.  Using viewsets is less explicit than building each view individually.

## Activities
- class-based views (activities before viewsets)
- Hyperlinked APIs
    - hyperlinkmodelserializer
- viewsets
- pagination?